In [1]:
#import sys
#!conda install --yes --prefix {sys.prefix} pygraphviz

# Kõik  deprel="amod" vahetult seotud  tipud  (korpuse tsv faili alusel)

## Vajalikud teegid

In [2]:
from __future__ import print_function

import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
from collections import defaultdict
import matplotlib.pyplot as plt
import random

#import etnltk
#from estnltk.storage.postgres import PostgresStorage

import networkx as nx
import sqlite3
import sys


## Abifunktsioonid

In [3]:
class graphFunctions:

    # kahe listi ühisosa 
    def intersection(a, b):
        return list(set(a).intersection(b))

    # tipu leidmine atribuudi väärtuse järgi
    def get_nodes_by_attributes(G,  attrname, attrvalue ):
        nodes = defaultdict(list)
        {nodes[v].append(k) for k, v in nx.get_node_attributes(G,attrname).items()}
        if attrvalue in nodes:
            return dict(nodes)[attrvalue]
        return []

    # graafi joonistamine 
    # tipp - lemma
    # serv - deprel
    def drawGraph(G):
        pos = graphviz_layout(G, prog='dot')
        labels = nx.get_node_attributes(G, 'lemma')
        nx.draw(G, pos, cmap = plt.get_cmap('jet'),labels=labels, with_labels=True)
        edge_labels = nx.get_edge_attributes(G, 'deprel')
        nx.draw_networkx_edge_labels(G, pos, edge_labels)
        plt.show()


In [4]:
def eprint(*args, **kwargs):
    print(*args, file=sys.stderr, **kwargs)

### Baasi  (sqlite3)  ettevalmistamine täitmiseks

Luuakse tabelid:
* collections_processed - salvestatakse viimane salvestatud collection Id
* {TABLENAME} - tabel kollokatsioonidega
 * lemma1 text
 * pos1 text
 * lemma2 text
 * pos2 text
 * total integer
 * example1 text
 * example2 text
 * example3 text

Luuakse indeksid:
* collections_processed_uniq
* {TABLENAME}_col1_col2_unique

In [5]:
def prepCollDb():
    global TABLENAME, cursor, conn

    cursor.execute(f"""CREATE TABLE IF NOT EXISTS collections_processed
                      (tablename text, lastcollection integer);
                      """)
    
    cursor.execute(f"""CREATE UNIQUE INDEX IF NOT EXISTS collections_processed_uniq ON collections_processed(tablename);
    """)
    
    #tsv failist lugemise korral loome tabeli alati nullist
    cursor.execute(f"""
        INSERT INTO collections_processed VALUES (?,?)
        ON CONFLICT(tablename) DO UPDATE SET lastcollection=?;""", (TABLENAME, 0, 0,) )
    
 
    cursor.execute(f"""CREATE TABLE IF NOT EXISTS {TABLENAME}
                      (lemma1 text, pos1 text, lemma2 text, pos2 text, total integer, example1 text, example2 text, example3 text);
                      """)

    #tsv failist lugemise korral loome tabeli alati nullist
    cursor.execute(f"""DELETE FROM {TABLENAME};""")

    
    INDEXNAME = f'{TABLENAME}_col1_col2_unique'
    cursor.execute(f"""CREATE UNIQUE INDEX IF NOT EXISTS {INDEXNAME}
        ON {TABLENAME}(lemma1, pos1, lemma2, pos2);
        """)
    
    conn.commit()



Tulemuse salvestamine baasi (sqlite3)

In [6]:
def saveCollToDb(collocations, examples, lastcollection):
    
    global TABLENAME, cursor, conn
    sqlColls = []

    for key in collocations.keys():
        example1 = None
        example2 = None
        example3 = None

        if len(examples[key])>0:
            example1 = examples[key][0]
        if len(examples[key])>1:
            example2 = examples[key][1]
        if len(examples[key])>2:
            example3 = examples[key][2]

        sqlColls.append( (key[0], key[2], key[1],  key[3], collocations[key], example1, example2, example3 , collocations[key], example1, example2,example3,) )

    cursor.executemany(f"""
        INSERT INTO {TABLENAME} VALUES (?,?,?,?,?,?,?,?)
        ON CONFLICT(lemma1, pos1, lemma2, pos2)
        DO UPDATE SET total=total+?
            ,  example1= ?
            ,  example2= ?
            ,  example3= ?
            ;""", sqlColls)

    cursor.execute(f"""
        INSERT INTO collections_processed VALUES (?,?)
        ON CONFLICT(tablename) DO UPDATE SET lastcollection=?;""", (TABLENAME, lastcollection, lastcollection,) )
    
    conn.commit()
    eprint(f'andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - {lastcollection}' )
    

### advmod - sõltuvuste leidmine 

* Kõik paarid, mille vahel on deprel = advmod

#### Näide 1

autor --> eelkõige

<img src="img/all_advmod_01.png" width="600" />

#### Näide 2

palju --> võimalikult

müra --> palju

<img src="img/all_advmod_02.png" width="600" />


In [7]:
def extract_advmod(G, collocations, examples, sentence):
        # lyhim tee tippude vahel
        path = nx.all_pairs_shortest_path_length(G)
        path_reversed = nx.all_pairs_shortest_path_length(G.reverse())

        # kauguste maatriksid
        dpath = {x[0]:x[1] for x in path}
        dpath_reversed = {x[0]:x[1] for x in path}

        #eprint('eraldame tipud vajalike parameetritega ')
        advmod_nodes = graphFunctions.get_nodes_by_attributes(G,  attrname = 'deprel', attrvalue =  'advmod')

        for node in advmod_nodes:
            parents = [parent for parent in G.predecessors(node)]
            if len(parents)>1:
                print ('Mingi jama puu struktuuriga - tipul on rohkem kui üks vanem')
                synFunctions.drawGraph(G)
                exit(1)
            parent = parents[0]
            #print (G.nodes[parent]['lemma'], '-->', G.nodes[node]['lemma'])
            #graphFunctions.drawGraph(G, parent, node);
            try:
                key = ( G.nodes[parent]['lemma'], G.nodes[node]['lemma'], G.nodes[parent]['pos'],  G.nodes[node]['pos'] , )

            except:
                print('Mingi jama puu struktuuriga', node, parent)
                print (sentence )
                print (G.nodes)
                print (G.edges)
                continue

            if not key in collocations:
                collocations[key] = 0
            if not key in examples:
                examples[key] = []
            collocations[key] += 1
            examples[key].append(sentence)
            if len(examples[key]) > 3:
                del(examples[key][random.randint(0, 2)])
        return (collocations, examples)

## Muutujad

**Korpuse TSV fail**

Lähte TSV-faili ja DB tabeli nimi, kuhu tulemus salvestatakse (TSV genereerimise kood on failis **collect_texts_db_tsv.ipynb**)

In [8]:
collectionName = 'koondkorpus_base_subset_of_5000_v2' # 'koondkorpus_base_v2'
corporafile = f'{collectionName}.tsv'

**Kollokatsiooni tüüp**

In [9]:
TYPE = 'ALL_advmod'

**Tabeli nimi andmebaasis**

In [10]:
TABLENAME = f'{TYPE}_{collectionName}'

**Kollektsioonide arv**, mille kaupa salvestatakse vahepealne tulemus andmebaasi.

In [11]:
BATCHSIZE = 1000

## Andmebaasi loomine ja ette valmistamine

In [12]:
conn = sqlite3.connect(f"{TYPE}_collocations.db") #
cursor = conn.cursor()
prepCollDb()

## Kollokatsioonide leidmine

Andmed loetakse TSV failist. Igast lausest tehakse *networkx* suunatud graaf. Graafi servadeks on süntaksipuu 
head->child seos. Tippudeks on lause sõnad. 
Tipu omadused on:
* *id* - sõna id
* deprel
* lemma
* pos


In [13]:

unsaved = 0
collocations = {}
examples = {}
word_id = 0
count = 0
current_sentence = []

#
G = None
colId = None
prevCol = 0
with open(corporafile) as f:
    for line in f:
        count +=1
        line = line.strip()
        row = line.split('\t')
        if not len(row) == 8:
            continue
        data = {}
        prevCol = colId
        (colId, data['start'], data['id'], data['text'], data['lemma'], data['upostag'], data['deprel'], data['head']) = row
        colId = int(colId)
        for k in ('id', 'start', 'head'):
            data[k] = int(data[k])

        word_id +=1
        #sentence start
        if data['id'] == 1:
            if not G==None:
                current_sentence_text = ' '.join([s['text'] for s in current_sentence])
                (collocations, examples) = extract_advmod(G, collocations, examples, current_sentence_text)
                unsaved = 1
            current_sentence = []
            #count+=1
            word_id +=1
            G = nx.DiGraph()

            if not prevCol ==colId and not colId==0 and not colId%BATCHSIZE:
                saveCollToDb(collocations, examples, colId)
                collocations = {}
                unsaved = 0

        #paneme graafi kokku
        G.add_node(word_id, id=data['id'], lemma=data['lemma'], pos=data['upostag'], deprel=data['deprel'])
        G.add_edge(word_id - data['id'] + data['head'], word_id, deprel = data['deprel'])
        current_sentence.append(data)
    
    #viimane lause
    current_sentence_text = ' '.join([s['text'] for s in current_sentence])
    (collocations, examples) = extract_advmod(G, collocations, examples, current_sentence_text)
    saveCollToDb(collocations, examples, colId)


andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 1000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 2000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 3000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 4000
andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 4999


### Andmebaasi indeksite lisamine


Tehakse kõige viimasena, et andmete sisestamine andmebaasi oleks kiirem.

In [14]:

indexesQ = [
    f'CREATE INDEX IF NOT EXISTS "lemma1" ON "{TABLENAME}" ("lemma1");'
    , f'CREATE INDEX IF NOT EXISTS "lemma2" ON "{TABLENAME}" ("lemma2");'
    , f'CREATE INDEX IF NOT EXISTS "pos1" ON "{TABLENAME}" ("pos1");'
    , f'CREATE INDEX IF NOT EXISTS "pos2" ON "{TABLENAME}" ("pos2");'
    , f'CREATE INDEX IF NOT EXISTS "total" ON "{TABLENAME}" ("total" DESC);']

for q in indexesQ: cursor.execute(q)

In [15]:
cursor.execute(f"SELECT count(*) FROM {TABLENAME}")
all_collocations = cursor.fetchall()

In [16]:
all_collocations

[(78146,)]